In [1]:
import pandas as pd
import plotly.express as px

In [11]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['edition','result_id','athlete','athlete_id','pos'])


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1



In [12]:
athlete_results.drop(repeat, inplace=True)

In [13]:
athlete_results.drop(columns=['edition_id', 'isTeamSport'], inplace=True)

In [14]:
yn_medal = []
for medal in athlete_results['medal']:
    if medal == None:
        yn_medal.append("No medal")
    else:
        yn_medal.append("With medal")
athlete_results['with_medal'] = yn_medal

In [15]:
country = 'USA'
gr = pd.DataFrame(athlete_results.groupby(['country_noc', 'sport', 'with_medal']).count()).reset_index()

gr.rename(columns={"event": "count"}, inplace=True)

new_gr = gr[gr['country_noc'] == 'USA']

display(new_gr)

,country_noc,sport,with_medal,index,count,medal
5165,USA,3x3 Basketball,With medal,1,1,1
5166,USA,American Football,No medal,2,2,0
5167,USA,Archery,No medal,109,109,0
5168,USA,Archery,With medal,31,31,31
5169,USA,Art Competitions,No medal,520,520,0
...,...,...,...,...,...,...
5274,USA,Weightlifting,No medal,117,117,0
5275,USA,Weightlifting,With medal,46,46,46
5276,USA,Wrestling,No medal,262,262,0
5277,USA,Wrestling,With medal,142,142,142


In [16]:
fig = px.sunburst(new_gr, path=['sport', 'with_medal'], values='count')
fig.show()

In [17]:
athlete_Bio = pd.read_csv('Dataset/Olympic_Athlete_Bio.csv', sep=',')
athlete_Bio = athlete_Bio.replace(["na"], None)
athlete_Bio["height"] = athlete_Bio["height"].astype(float)

athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)

In [18]:
a_bio = {'athelete_id': athlete_Bio['athlete_id'], 'sex': athlete_Bio['sex']}
a_event = {'athelete_id': athlete_Event_Results['athlete_id'], 'medal': athlete_Event_Results['medal']}

df = pd.merge(pd.DataFrame(a_bio), pd.DataFrame(a_event), on='athelete_id')

count = pd.DataFrame(df.groupby(['sex']).count())
count = count['athelete_id']

df1 = df.dropna(subset=['medal'])

print(count)

perc = pd.DataFrame(df1.groupby(['sex', 'medal']).count()).reset_index()


perc.loc[perc["sex"] == "Male", "athelete_id"] = (perc['athelete_id']/count[1])*100
perc.loc[perc["sex"] == "Female", "athelete_id"] = (perc['athelete_id']/count[0])*100

perc.rename(columns={"athelete_id": "count"}, inplace=True)
print(perc)

fig = px.sunburst(perc, path=['medal', 'sex'], values='count', color_discrete_map={'Gold': '#D6AF36', 'Silver': '#A7A7AD', 'Bronze': '#A77044'})
fig.show()


sex
Female     89310
Male      225597
Name: athelete_id, dtype: int64
      sex   medal     count
0  Female  Bronze  5.040869
1  Female    Gold  5.002799
2  Female  Silver  4.969208
3    Male  Bronze  4.610877
4    Male    Gold  4.680470
5    Male  Silver  4.522223


In [19]:
a_bio = {'athelete_id': athlete_Bio['athlete_id'], 'sex': athlete_Bio['sex']}
a_event = {'athelete_id': athlete_Event_Results['athlete_id'], 'edition': athlete_Event_Results['edition']}
a_event['edition'] = a_event['edition'].str.split(' ')

a_event = {'athelete_id': a_event['athelete_id'], 'year': a_event['edition'].str[0], 'season': a_event['edition'].str[1]}

df = pd.merge(pd.DataFrame(a_bio), pd.DataFrame(a_event), on='athelete_id')

count = pd.DataFrame(df.groupby(['year', 'sex', 'season']).count()).reset_index()
count.rename(columns={"athelete_id": "count"}, inplace=True)

print(count)

summer = count[count['season'] == 'Summer']
winter = count[count['season'] == 'Winter']

fig = px.sunburst(summer, path=['year', 'sex'], values='count')
fig.show()

fig = px.sunburst(winter, path=['year', 'sex'], values='count')
fig.show()

     year     sex  season  count
0    1896  Female  Summer      2
1    1896    Male  Summer    617
2    1900  Female  Summer     53
3    1900    Male  Summer   3094
4    1904  Female  Summer     17
..    ...     ...     ...    ...
104  2018    Male  Winter   3036
105  2020  Female  Summer   7126
106  2020    Male  Summer   7660
107  2022  Female  Winter   2430
108  2022    Male  Winter   2958

[109 rows x 4 columns]
